In [14]:
import firebase_admin
import pandas as pd
from firebase_admin import credentials, firestore
from datetime import datetime, timedelta, timezone

In [13]:
!pip install pandas

  Using cached pandas-2.3.3-cp39-cp39-macosx_11_0_arm64.whl.metadata (91 kB)
  Using cached numpy-2.0.2-cp39-cp39-macosx_14_0_arm64.whl.metadata (60 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
Using cached pandas-2.3.3-cp39-cp39-macosx_11_0_arm64.whl (10.8 MB)
Using cached numpy-2.0.2-cp39-cp39-macosx_14_0_arm64.whl (5.3 MB)
Using cached pytz-2025.2-py2.py3-none-any.whl (509 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4/4 [pandas]2m3/4 [pandas]


In [6]:
TENANT_ID = "vWxkIvCXIJUSIlhBSiIl"


def init_firebase() -> firestore.Client:
    try:
        cred = credentials.Certificate("/Users/marciomacedo/dev/github/pscripts/key-prod.json")
        firebase_admin.initialize_app(cred)
        print("Firebase app initialized successfully")
    except ValueError as e:
        if "already exists" in str(e):
            print("Firebase app already initialized, continuing...")
        else:
            raise

    return firestore.client()

In [18]:
    db = init_firebase()

    now = datetime.now(timezone.utc)
    start = now - timedelta(days=4)

Firebase app already initialized, continuing...


In [19]:
    report_col = (
        db.collection("Tenant")
        .document(TENANT_ID)
        .collection("Forms")
        .document("audit")
        .collection("report")
    )

    query = (
        report_col.where("createdIn", ">=", start)
        .where("createdIn", "<=", now)
        .order_by("createdIn")
    )


/Users/marciomacedo/dev/github/pscripts/.venv/lib/python3.9/site-packages/google/cloud/firestore_v1/base_collection.py:315: UserWarning: Detected filter using positional arguments. Prefer using the 'filter' keyword argument instead.
  return query.where(field_path, op_string, value)
/var/folders/ld/904s82bn71s7yz3vk7fhzktm0000gn/T/ipykernel_96889/4035084647.py:10: UserWarning: Detected filter using positional arguments. Prefer using the 'filter' keyword argument instead.
  report_col.where("createdIn", ">=", start)


In [20]:
    count = 0
    rows = []
    for doc in query.stream():
        data = doc.to_dict() or {}
        created_in = data.get("createdIn")
        user_cpf = data.get("userCpf")
        user_id = data.get("userId")
        classification = data.get("classification")

        print(
            f"- id={doc.id} createdIn={created_in} userId={user_id} userCpf={user_cpf} classification={classification}"
        )
        rows.append(
            {
                "id": doc.id,
                "createdIn": created_in,
                "userId": user_id,
                "userCpf": user_cpf,
                "classification": classification,
            }
        )
        count += 1

- id=1OoApIUmvpfDWv6QYVnjp9otk713 createdIn=2026-01-25 17:00:34.577000+00:00 userId=1OoApIUmvpfDWv6QYVnjp9otk713 userCpf=54765564860 classification=zona_2
- id=MrGarTc2w4TwbCsBEdWBar367yt2 createdIn=2026-01-26 01:26:52.025000+00:00 userId=MrGarTc2w4TwbCsBEdWBar367yt2 userCpf=55101827843 classification=zona_1_em_abstinencia
- id=KaLGWUpEMlVNlQsSF6h4O7SayZf2 createdIn=2026-01-26 01:31:20.831000+00:00 userId=KaLGWUpEMlVNlQsSF6h4O7SayZf2 userCpf=45581325812 classification=zona_1_com_bpe
- id=zKK08QgNldQv8O2ZPVY2NbdlhYy1 createdIn=2026-01-26 02:15:56.794000+00:00 userId=zKK08QgNldQv8O2ZPVY2NbdlhYy1 userCpf=23080144821 classification=zona_1_em_abstinencia
- id=Z0yRH9ZKfyfk7jySSNz05Es4Txd2 createdIn=2026-01-26 02:16:40.329000+00:00 userId=Z0yRH9ZKfyfk7jySSNz05Es4Txd2 userCpf=24957839883 classification=zona_1_em_abstinencia
- id=2HFhvW5LwTaA9FcHdTfbO05nHzS2 createdIn=2026-01-26 11:45:26.502000+00:00 userId=2HFhvW5LwTaA9FcHdTfbO05nHzS2 userCpf=10719918502 classification=zona_4
- id=JFKODnXdSLT0

In [21]:

df = pd.DataFrame(rows)

In [23]:
zona_1_sem_bpe = df[df['classification'] == 'zona_1_sem_bpe']
zona_1_sem_bpe.to_csv('zona_1_sem_bpe.csv', index=False)
